# BARTScore
* github : https://github.com/neulab/BARTScore

동작과정

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.1 MB/s eta 0:00:00


In [10]:
import torch
import torch.nn as nn
import traceback
from transformers import BartTokenizer, BartForConditionalGeneration
from tqdm import tqdm

In [3]:
device='cuda:0'
max_length = 1024
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((102

In [5]:
loss_fct = nn.NLLLoss(reduction='none', ignore_index=model.config.pad_token_id)
lsm = nn.LogSoftmax(dim=1)

## Score

In [23]:
def score(srcs, tgts, batch_size=4):
        """ Score a batch of examples """
        score_list = []
        for i in tqdm(range(0, len(srcs), batch_size)):
            src_list = srcs[i: i + batch_size]
            tgt_list = tgts[i: i + batch_size]
            try:
                with torch.no_grad():
                    encoded_src = tokenizer(
                        src_list,
                        max_length=max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    encoded_tgt = tokenizer(
                        tgt_list,
                        max_length=max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='pt'
                    )
                    src_tokens = encoded_src['input_ids'].to(device)
                    src_mask = encoded_src['attention_mask'].to(device)
                    print(f'src_tokens {src_tokens.shape} : {src_tokens}')

                    tgt_tokens = encoded_tgt['input_ids'].to(device)
                    tgt_mask = encoded_tgt['attention_mask']
                    tgt_len = tgt_mask.sum(dim=1).to(device)
                    print(f'tgt_tokens {tgt_tokens.shape} : {tgt_tokens}')

                    output = model(
                        input_ids=src_tokens,
                        attention_mask=src_mask,
                        labels=tgt_tokens
                    )
                    logits = output.logits.view(-1, model.config.vocab_size)
                    print(f'logits {logits.shape} : {logits}')
                    print(f'AFTER NLL : {lsm(logits)}')
                    print(f'tgt_tokens.view : {tgt_tokens.view(-1)}')

                    loss = loss_fct(lsm(logits), tgt_tokens.view(-1))
                    loss = loss.view(tgt_tokens.shape[0], -1)
                    loss = loss.sum(dim=1) / tgt_len
                    print(f'loss : {loss}')

                    curr_score_list = [-x.item() for x in loss]
                    score_list += curr_score_list

            except RuntimeError:
                traceback.print_exc()
                print(f'source: {src_list}')
                print(f'target: {tgt_list}')
                exit(0)
        return score_list

## Test sample

In [24]:
""" Test """
src_list = ['This is a very good idea. Although simple, but very insightful.','Can I take a look?','Do not trust him, he is a liar.']
tgt_list = ["That's stupid.", "What's the problem?", 'He is trustworthy.']

In [25]:
score(src_list, tgt_list)

100%|██████████| 1/1 [00:00<00:00, 18.42it/s]

src_tokens torch.Size([3, 16]) : tensor([[    0,   713,    16,    10,   182,   205,  1114,     4,  2223,  2007,
             6,    53,   182, 29804,     4,     2],
        [    0, 10836,    38,   185,    10,   356,   116,     2,     1,     1,
             1,     1,     1,     1,     1,     1],
        [    0,  8275,    45,  2416,   123,     6,    37,    16,    10, 28587,
             4,     2,     1,     1,     1,     1]], device='cuda:0')
tgt_tokens torch.Size([3, 7]) : tensor([[    0,  1711,    18, 12103,     4,     2,     1],
        [    0,  2264,    18,     5,   936,   116,     2],
        [    0,   894,    16, 32101,     4,     2,     1]], device='cuda:0')
logits torch.Size([21, 50264]) : tensor([[10.8939,  0.5972,  2.8265,  ...,  0.2649,  0.3191,  0.1089],
        [10.8939,  0.5972,  2.8265,  ...,  0.2649,  0.3191,  0.1089],
        [-3.8713, -0.3617,  2.4232,  ..., -0.8573, -0.8283, -0.8292],
        ...,
        [-1.5935, -0.3027,  2.5608,  ..., -0.3889, -0.5389, -0.6070],
   

[-4.3136420249938965, -3.534801483154297, -4.1931071281433105]